# Object Segmenation on Azure Stack Hub Clusters

For this tutorial, we will be finetuning a pre-trained [Mask R-CNN](https://arxiv.org/abs/1703.06870) model in the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/). Ithttps://www.cis.upenn.edu/~jshi/ped_html/contains 170 images with 345 instances of pedestrians, and we will use it to illustrate how to use the new features in torchvision in order to train an instance segmentation model on a custom dataset.

In [ ]:
import os
from azureml.core import Workspace,Environment, Experiment, Datastore
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

## Create Workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [ ]:
ws = Workspace.from_config()

## Register Dataset

After downloading and extracting the zip file from [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/) to your local machine, you will have the following folder structure:

<pre>
PennFudanPed/
  PedMasks/
    FudanPed00001_mask.png
    FudanPed00002_mask.png
    FudanPed00003_mask.png
    FudanPed00004_mask.png
    ...
  PNGImages/
    FudanPed00001.png
    FudanPed00002.png
    FudanPed00003.png
    FudanPed00004.png
</pre>

### todo: reference: how to create an datast

In [ ]:
from azureml.core import Workspace, Dataset, Datastore

dataset_name = "pennfudan_2"
if dataset_name not  in ws.datasets:
    
    datastore =  Datastore.get(ws, "ashstore")
    datastore.upload('PennFudanPed', 'PennFudanPed')

    datastore_paths = [(datastore, 'PennFudanPed')]

    pd_ds = Dataset.File.from_files(path=datastore_paths)
    pd_ds.register(ws, dataset_name, "for Pedestrian Detection and Segmentation")

## Create Compute Target

In [ ]:
resource_id = "<resource_id>"
attach_name = "slw4pipeline11"
script = 'obj_segament.py'

attach_config = ArcKubernetesCompute.attach_configuration(resource_id=resource_id)

attach_result = ArcKubernetesCompute.attach(ws, attach_name, attach_config)

attach_result.wait_for_completion(show_output=True)

print(attach_result)

compute_target = ws.compute_targets[attach_name]


## Create a Training-Test split data process Step

For this pipeline run, you will use two pipeline steps.  The first step is to split dataset into training and testing.

In [ ]:
# create run_config first

env = Environment.from_dockerfile(
        name='pytorch-obj-seg',
        dockerfile='./aml_src/Dockerfile.gpu',
        conda_specification='./aml_src/conda-env.yaml')

aml_run_config = RunConfiguration()
aml_run_config.target = compute_target.name
aml_run_config.environment = env

source_directory = './aml_src'


# add a data process step

dataset = ws.datasets[dataset_name]

from azureml.data import OutputFileDatasetConfig

dest = (datastore, None)

train_split_data = OutputFileDatasetConfig(name="train_split_data", destination=dest).as_upload(overwrite=False)
test_split_data = OutputFileDatasetConfig(name="test_split_data", destination=dest).as_upload(overwrite=False)


split_step = PythonScriptStep(
    name="Train Test Split",
    script_name="obj_segment_step_data_process.py",
    arguments=["--data-path", dataset.as_named_input('pennfudan_data').as_mount(),
               "--train-split", train_split_data, "--test-split", test_split_data,
               "--test-size", 50],
    compute_target=compute_target,
    runconfig=aml_run_config,
    source_directory=source_directory,
    allow_reuse=False
)

## Create Training Step

In [ ]:
train_step = PythonScriptStep(
        name="training_step",
        script_name="obj_segment_step_training.py",
        arguments=[
            "--train-split", train_split_data.as_input(), "--test-split", test_split_data.as_input(),
            '--epochs', 1,  # 80
        ],

        compute_target=compute_target,
        runconfig=aml_run_config,
        source_directory=source_directory,
        allow_reuse=True
    )
    

## Create Experiment and Submit Pipeline Run

In [ ]:
experiment_name = 'obj_seg_step'
experiment = Experiment(workspace=ws, name=experiment_name)
pipeline_steps = [train_step]

pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

pipeline_run = experiment.submit(pipeline, regenerate_outputs=False)
pipeline_run.wait_for_completion()


## Register Model

In [ ]:
train_step_run = pipeline_run.find_step_run(train_step.name)[0]

model_name = 'obj_seg_model_2'
train_step_run.register_model(model_name=model_name, model_path='outputs/obj_segmentation.pkl')

## Deploy the Model

### Imports

In [ ]:
from azureml.core import Environment, Workspace, Model, ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.compute_target import ComputeTargetException
from PIL import Image
from torchvision.transforms import functional as F
import numpy as np
import json

### Create a AKS cluster for serving the model

In [ ]:
ws = Workspace.from_config()

# Choose a name for your AKS cluster
aks_name = 'aks-service-2'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

### Deploy the model

In [ ]:
env = Environment.from_dockerfile(
        name='pytorch-obj-seg',
        dockerfile='./aml_src/Dockerfile.gpu',
        conda_specification='./aml_src/conda-env.yaml')

env.inferencing_stack_version='latest'

inference_config = InferenceConfig(entry_script='score.py', environment=env)
deploy_config = AksWebservice.deploy_configuration()

deployed_model = "obj_segmentation_model" # model_name
model = ws.models[deployed_model]

service_name = 'objservice2'

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deploy_config,
                       deployment_target=aks_target,
                       overwrite=True)

service.wait_for_deployment(show_output=True)


## Test Service

In [ ]:
img_nums = ["00001"]
image_paths = ["PennFudanPed\\PNGImages\\FudanPed{}.png".format(item) for item in img_nums]
image_np_list = []
for image_path in image_paths:
    img = Image.open(image_path)
    
    img.show()
    
    img_rgb = img.convert("RGB")
    img_tensor = F.to_tensor(img_rgb)
    img_np = img_tensor.numpy()
    image_np_list.append(img_np.tolist())

inputs = json.dumps(image_np_list)

resp = service.run(inputs)

predicts = resp["predicts"]
p_str = json.dumps(predicts)

p_obj = json.loads(p_str)

# put the model in evaluation mode
for image_data in p_obj:
    img_np = np.array(image_data)
    output = Image.fromarray(img_np)
    output.show()

### Create a  function to call the url end point

In [ ]:
import urllib.request
import json

from PIL import Image
from torchvision.transforms import functional as F
import numpy as np


def service_infer(url, body, api_key):
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        return result

    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(json.loads(error.read().decode("utf8", 'ignore')))

## Testing

The deployed service endpoints will be shown in workspace's EndPoints section. You may find the REST url and apiKey in  the consume part of the endpoint.

In [ ]:
url = 'http://13.90.195.160:80/api/v1/service/objservice/score'
api_key = 'hSySjdRpcqXLXqVyd7V5e1oaRUEbXhpu'  # Replace this with the API key for the web service

img_nums = ["00001","00002"]
image_paths = ["PennFudanPed\\PNGImages\\FudanPed{}.png".format(item) for item in img_nums]
image_np_list = []
for image_path in image_paths:
    img = Image.open(image_path)
    img.show()
    img_rgb = img.convert("RGB")
    img_tensor = F.to_tensor(img_rgb)
    img_np = img_tensor.numpy()
    image_np_list.append(img_np.tolist())

inputs = json.dumps(image_np_list)

body = str.encode(inputs)
resp = service_infer(url, body, api_key)

# predicts = resp["predicts"]
# p_str = json.dumps(predicts)

p_obj = json.loads(resp)

# put the model in evaluation mode
for image_data in p_obj["predicts"]:
    img_np = np.array(image_data)
    output = Image.fromarray(img_np)
    output.show()
